In [1]:
from oddsapi import OddsApiClient
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime as dt
import os
from dotenv import load_dotenv
load_dotenv()

MYDIR = ("odds")


import statsapi




In [ ]:
from enum import Enum
class SportsBooks(Enum):
    bet_online = "betonlineag"
    barstool = "barstool"
    betfair = "betfair"
    betmgm =  "betmgm"
    betrivers = "betrivers"
    betus = "betus"
    bovada = "bovada"
    draftkings = "draftkings"
    fanduel = "fanduel"
    foxbet = "foxbet"
    gtbet = "gtbets"
    intertops = "intertops"
    lowwig = "lowvig"
    mybookie = "mybookieag"
    pointsbet = "pointsbetus"
    sugarhouse = "sugarhouse"
    twinspires = "twinspires"
    unibet = "unibet"
    williamhill = "williamhill_us"
    wynnbet = "wynnbet"



In [ ]:
url = "https://api.the-odds-api.com"
ext = "/v3/sports/"
params = {"apiKey": key}
r = requests.get(url=url + ext, params=params)
r.status_code

In [ ]:
r.content
sports = json.loads(r.text)
sports['data'][0]
for i in sports['data']:
    print(i['title'], i['active'])

In [ ]:

url = "https://api.the-odds-api.com"
req = f"/v3/odds/"
params = {
    "apiKey": key,
    "sport" :'basketball_nba',
"region" :'us',
'mkt':'h2h',
'oddsFormat': 'american' 
}
odds_req = requests.get(url + req, params=params)

In [ ]:
odds = json.loads(odds_req.text)
odds = odds['data']
# odds = odds[0]
# arb = []
# for game in odds:
#     if dt.now() > dt.fromtimestamp(game['commence_time']):
#         continue
#     g = Game(game)
#     if g.arb_exists():
#         print(f"Arb exists for {g.away} @ {g.home} at {g.start_time}! Secure that bag!")
#         arb.append(game)
# print(len(odds))
# for game in arb:
#     print(game.beat_bookies())
odds


In [ ]:
def get_all_odds(sites, home_first = True,draw_possible=False):
    '''
    Given the sites value from an Odds-API request
    return a formatted dict of all of the odds for logging
    Inputs:
        sites: list of odds from various dicts of sports betting odds
        draw_possible: (bool) boolean to denote whether the sport has draws in H2H odds
                        (not currently supported) 
    '''
    odds = {}
    for site in sites:
        name = site['site_key']
        last_update = site['last_update']
        line = site['odds']['h2h']
        if home_first:
            home_odds = line[0]
            away_odds = line[1]
        else:
            home_odds = line[1]
            away_odds = line[0]
        odds[name + '_home'] = home_odds
        odds[name + '_away'] = away_odds
        odds[name + '_last_update'] = dt.fromtimestamp(last_update)
    return odds

get_all_odds(odds[0]['sites'])

In [ ]:
class OddsLogger(object):
    """
    Object to automatically odds for backtesting purposes.
    Given a list of games, formats those whose start_time is within an 
    hour of the current time into a .csv file under the appropriate diretory
    current time and 

    Args:
        sport (str): the sport to log the odds on the sportsbooks. Development to come
    """
    def __init__(self, sport):
        self.__api_key = os.getenv('API_KEY')
        base_url = "https://api.the-odds-api.com"
        odds_endpoint = f"/v3/odds/"
        params = {
            "apiKey": self.__api_key,
            "sport" :'baseball_mlb',
            "region" :'us',
            'mkt':'h2h',
            'oddsFormat': 'american'
        }
        odds_req = requests.get(url + req, params=params)
        odds = json.loads(odds_req.text)
        odds = odds['data']
        self.games = []
        for game in odds:
            if dt.fromtimestamp(game['commence_time']) < dt.now(): #Ignore live odds
                continue
            row = pd.Series()
            row["ID"] = game['id']
            row['Sport'] = game['sport_nice']
            row['Home'] = game['home_team']
            row['Away'] = [x for x in game['teams'] if x != game['home_team']][0]
            row['Start Time'] = dt.fromtimestamp(game['commence_time'])
            home_first = True if game['teams'][0] == game['home_team'] else False
            odds_by_sb = get_all_odds(game['sites'], home_first, False)
            books_quoting = [s['site_key'] for s in game['sites']]
            for book in SportsBooks:
                if book.name in books_quoting:
                    row[f'{book.name}_last_update'] = odds_by_sb[f"{book.name}_last_update"]
                    row[f"{book.name}_home"] = odds_by_sb[f"{book.name}_home"]
                    row[f"{book.name}_away"] = odds_by_sb[f"{book.name}_away"]
                else:
                    row[f'{book.name}_last_update'] = np.nan
                    row[f"{book.name}_home"] = np.nan
                    row[f"{book.name}_away"] = np.nan
            self.games.append(row)
        self.odds_frame = pd.DataFrame(self.games).set_index("ID")
        self.odds_by_month = self.split_months()
        self.merge_with_existing_odds()


    def get_all_odds(self, sites, home_first = True,draw_possible=False):
        '''
        Given the sites value from an Odds-API request
        return a formatted dict of all of the odds for logging
        Inputs:
            sites: list of odds from various dicts of sports betting odds
            draw_possible: (bool) boolean to denote whether the sport has draws in H2H odds
                            (not currently supported) 
        '''
        odds = {}
        for site in sites:
            name = site['site_key']
            last_update = site['last_update']
            line = site['odds']['h2h']
            if home_first:
                home_odds = line[0]
                away_odds = line[1]
            else:
                home_odds = line[1]
                away_odds = line[0]
            odds[name + '_home'] = home_odds
            odds[name + '_away'] = away_odds
            odds[name + '_last_update'] = last_update
            if draw_possible:
                odds[name + '_draw'] = line[-1]
        return odds

    def split_months(self):
        g = self.odds_frame.groupby(pd.Grouper(key="Start Time", freq="M"))
        return [group for _,group in g]

    def merge_with_existing_odds(self):
        """
        Iterates through dataframes in self.odds_by_month and merges current
        data with existing data prioritizing the most current lines
        """
        if not os.path.isdir(MYDIR):
            os.makedirs(MYDIR)
        for m in self.odds_by_month:
            month = m['Start Time'][0].strftime("%B")
            year = m['Start Time'][0].year
            path = MYDIR + "\\" + month + "_" + str(year) + ".csv"
            if not os.path.exists(path):
                m.to_csv(path)
            else:
                df = pd.read_csv(path, index_col="ID")
                df = pd.concat([df,m], axis = 0)
                df = df[~df.index.duplicated(keep='last')]
                df.to_csv(path)

        
o = OddsLogger('baseball_mlb')
o.odds_frame
        

In [ ]:
pd.DataFrame(o.games).to_csv('test.csv')

In [ ]:
from fractions import Fraction
from sympy import symbols, Eq, solve
MY_SPORTSBOOKS = ['Barstool Sportsbook', 'DraftKings', "BetMGM", 'BetRivers', 'FanDuel', 'PointsBet (US)', 'William Hill (Ceasars)']

class Game(object):
    '''
    Object used to organize data from OddsAPI client 
    for processing later
    Inputs:
        data - (dict) data from OddsAPI response. 
        Must include: 'sports_key', 'teams', 'home_team', 
                    'sites'

    '''
    def __init__(self, data):
        self.sport = data['sport_nice']
        self.teams = data['teams']
        self.home = data['home_team']
        for team in self.teams:
            if team != self.home:
                self.away = team
        self.h2h_odds = {}
        for site in data['sites']:
            site_name = site['site_nice']
            self.h2h_odds[site_name] = site['odds']['h2h']
        # self.h2h_odds = pd.DataFrame(self.h2h_odds).T
        # self.h2h_odds.columns = self.teams
        self.start_time = dt.fromtimestamp(int(data['commence_time']))
        self.num_sites = ['sites_count']
        self.best_bettable_odds()
    
    def __repr__(self):
        return f"Game object for {self.away} @ {self.home} on {self.start_time}"

    def best_bettable_odds(self):
        '''
        Iterates through the odds and finds the best odds on both sides 
        Currently only supports games with no draws
        '''
        best_team1 = None
        best_team1_site = None
        best_team2 = None
        best_team2_site = None
        for site, odds in self.h2h_odds.items():
            if site in MY_SPORTSBOOKS:
                if not best_team1:
                    best_team1 = odds[0]
                    best_team1_site = site
                    best_team2 = odds[1]
                    best_team2_site = site
                else:
                    if odds[0] > best_team1:
                        best_team1 = odds[0]
                        best_team1_site = site
                    if odds[1] > best_team2:
                        best_team2 = odds[1]
                        best_team2_site = site
                self.best_odds = {
                    f"{self.teams[0]}": (best_team1, best_team1_site),
                    f"{self.teams[1]}": (best_team2, best_team2_site)
                }
        # print(self.best_odds)

    def arb_exists(self):
        sure_bet = 0
        for odds in self.best_odds.values():
            d_odds = Game.decimal_odds(odds[0])
            sure_bet += 1 / d_odds
        if sure_bet < 1:
            return True
        else:
            return False

    @staticmethod
    def decimal_odds(odds: int) -> float:
        """
        :param odds: Integer (e.g., -350) or String (e.g., '3/1' or '5/4').
        :return: Float. Odds expressed in Decimal terms.
        """
        if isinstance(odds, float):
            return odds

        elif isinstance(odds, int):
            if odds >= 100:
                return abs(1 + (odds / 100))
            elif odds <= -101 :
                return 100 / abs(odds) + 1
            else:
                return float(odds)

        elif "/" in odds:
            odds = Fraction(odds)
            return round((odds.numerator / odds.denominator) + 1, 2)
        
    def beat_bookies(self, total_stake):
        odds1 = self.best_odds[self.teams[0]][0]
        odds2 = self.best_odds[self.teams[1]][0]
        x, y = symbols('x y')
        eq1 = Eq(x + y - total_stake, 0) # total_stake = x + y
        eq2 = Eq((odds2*y) - odds1*x, 0) # odds1*x = odds2*y
        stakes = solve((eq1,eq2), (x, y))
        total_investment = stakes[x] + stakes[y]
        profit1 = odds1*stakes[x] - total_stake
        profit2 = odds2*stakes[y] - total_stake
        print(type(profit1), type(total_investment))
        benefit1 = f'{round(profit1 / total_investment * 100,2)}%'
        benefit2 = f'{round(profit2 / total_investment * 100,2)}%'
        dict_gabmling = {f'{self.teams[0]} stake':stakes[x], f'{self.teams[1]} stake':stakes[y], 'Profit1':profit1, 'Profit2':profit2,
                        'Benefit1': benefit1, 'Benefit2': benefit2}
        return dict_gabmling

        

g = Game(odds)
g.beat_bookies(500)

In [ ]:
g.h2h_odds

In [ ]:
import mlbgame

day = mlbgame.day(2022, 6, 12, home = "Cubs", away="Cubs")
# game = day[0]
# game
day

In [ ]:
from pybaseball import schedule_and_record
data = schedule_and_record(2022, 'NYY')
# data.to_csv('test.csv')
data.head()

In [ ]:

from sportsreference.mlb.boxscore import Boxscore

uri = 'PIT/PIT202206230'
game_data = Boxscore("2022-06-")


game_data.dataframe

In [ ]:
df = pd.read_csv("mlb_odds\\June_2022.csv", index_col = "ID")
df.head()
for 

In [ ]:
pd.read_html('https://www.baseball-reference.com/teams/MIA/2022-schedule-scores.shtml#team_schedule')

In [ ]:
team = 'MIA'
link = f'https://www.baseball-reference.com/teams/{team}/2022-schedule-scores.shtml#team_schedule'

In [11]:
df = pd.read_html(link,index_col='Gm#')[0]
# df = df[['Date', 'Tm', 'Opp' ]]
df.head(15)

,Date,Unnamed: 2,Tm,Unnamed: 4,Opp,W/L,R,RA,Inn,W-L,...,GB,Win,Loss,Save,Time,D/N,Attendance,cLI,Streak,Orig. Scheduled
Gm#,,,,,,,,,,,,,,,,,,,,,
1,"Friday, Apr 8",boxscore,MIA,@,SFG,L-wo,5,6,10,0-1,...,1.5,Alvarez,Bass,NaN,3:26,D,40853,.98,-,NaN
2,"Saturday, Apr 9",boxscore,MIA,@,SFG,W,2,1,NaN,1-1,...,1.5,Okert,McGee,Bender,2:57,D,38885,.95,+,NaN
3,"Sunday, Apr 10",boxscore,MIA,@,SFG,L,2,3,NaN,1-2,...,1.5,Garcia,Rogers,Leone,2:43,D,37332,.94,-,NaN
4,"Monday, Apr 11",boxscore,MIA,@,LAA,L,2,6,NaN,1-3,...,2.0,Lorenzen,Hernandez,NaN,2:42,N,20480,.89,--,NaN
5,"Tuesday, Apr 12",boxscore,MIA,@,LAA,L-wo,3,4,NaN,1-4,...,2.5,Iglesias,Bender,NaN,2:59,D,16132,.87,---,NaN
6,"Thursday, Apr 14",boxscore,MIA,NaN,PHI,W,4,3,NaN,2-4,...,2.5,Alcantara,Gibson,Bender,3:23,N,31184,.91,+,NaN
7,"Friday, Apr 15",boxscore,MIA,NaN,PHI,W,7,1,NaN,3-4,...,2.5,Lopez,Eflin,NaN,3:15,N,11990,.93,++,NaN
8,"Saturday, Apr 16",boxscore,MIA,NaN,PHI,L,3,10,NaN,3-5,...,2.5,Suarez,Rogers,NaN,3:23,N,13412,.94,-,NaN
9,"Sunday, Apr 17",boxscore,MIA,NaN,PHI,W,11,3,NaN,4-5,...,2.5,Hernandez,Wheeler,NaN,3:32,D,11476,.93,+,NaN


In [2]:
from odds_logger import DiscordAlert

In [3]:
DiscordAlert("test again")